## [SOLUTION] Supervised Learning with EHR Data

This is a mostly filled in version of the workshop presented on 2/25/2022.  You'll still need to fill in our sunet_id in various locations to write to proper tables, and you'll need to point to your local application_default_credentials.json.  Have not filled in a solution for any of the optional extra material. 

**Objective**: The goal of this workship is to enable you to quickly develop and evaluate a baseline model for an arbitrary clinical prediction task. In this workshop you'll train and evaluate models that predict whether various components of a CBC with Differential lab will result outside of their normal range. This walkthrough should empower you to replicate this kind of analysis on any arbitrary supervised learning task you may care about. 

### Modules
Developing and evaluating a clinical supervised learning model can be broken into three distinct tasks.  
1. **Cohort Construction**: You need some mechanism to define a cohort of interest and assign outcomes of interest (labels) to each observation (example). 
2. **Featurization**: You need a way to take your cohort and construct a feature matrix for each observation. 
3. **Modelling**: You need a mechanism to train and evaluate a predictive model given features and labels. 

Each of these tasks have been broken down into distinct modules in code you will be using throughout this workshop. In the following notebook we break each of these three tasks down in considerable detail, and provide starter code for you construct, train, and evaluate a supervised learning task from scratch.   

Before we start however, you'll need to run a few setup steps.

**Disclaimer** : Current version of the code only supports constructing feature matrices from the `shc_core` dataset. See bottom of this noteook for future TODOs in developing this out further.  Would be great to get this supporting `lpch_core` and `OMOP` versions of our data soon.

### 0.1 Install requried packages
Before beginning I'd recommend creating a new environemnt to avoid any conflicting package dependencies in your current environemnt.  If you don't know how to do this it's likely not a huge deal - you'll just want to know that the cell below will update a set of your python packages to specific versions (the most recent version as of 2/25/22).  See [here](https://docs.python.org/3/tutorial/venv.html) for info about python virtual environemnts, or [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html) for info about conda environements.

Below is the list of  packages you'll need installed to execute code in this workshop. Running the cell will install these packages into your current python environment.  This shouldn't take more than 2 minutes. 


In [1]:
!pip install google-cloud-bigquery==2.34.0
!pip install numpy==1.22.2
!pip install pandas==1.4.1
!pip install tqdm==4.62.3
!pip install scipy==1.8.0
!pip install scikit-learn==1.0.2
!pip install pandas-gbq==0.17.1

### 0.2 Add medinfo.dataconversion to your system path
Assuming you haven't installed CDSS as a package in your python environement, you'll need to add the `medinfo.dataconversions` directory to your system path so that you can import python modules from it. To do this, execute the code below.  We'll be importanting code from three python files in this directory
1. `cohorts.py`
2. `featurizers.py`
3. `trainers.py`

In [2]:
import sys
sys.path.insert(1, '../../../medinfo/dataconversion/') # I'm assuming you haven't moved the location of this notebook

### 0.3 Point  GOOGLE_APPLICATION_CREDENTIALS to your application_default_credentials.json
Assuming you've already followed steps [here](https://github.com/HealthRex/CDSS/wiki/Dev-Environment-and-Google-BigQuery-Database-Access-Setup#2-set-an-environment-variable-so-your-code-knows-where-to-find-this-key-file) to create an application_default_credentials.json, you'll need to set your GOOGLE_APPLICATION_CREDENTIALS environment variable so that is points to it. You can do this using the code snippet below, which may produce a warning that you can ignore. 

It's worth setting this env variable in your .bashrc / .bash_profile so that you don't have to do this each time you want to programmatically access our bigquery projects. 

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
    '/Your/Path/To/application_default_credentials.json'
)
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions'

## 1. Cohort Construction
Now onto the fun stuff. To develop and evaluate any clinical prediction model, you'll first need to define a patient cohort of interest. This is one of the more nuanced aspects of any predictive modelling task as it often requires a deep understanding of the underlying data (ex: our clarity extracts) to get right, and is task specific. Every time you embark on a new predictive modelling endeavor you need to design the cohort construction process essentially from scratch.  Some aspects of constructing a cohort will remain constant from task to task, and we'll leverage this fact to design a `CohortBuilder` class that can be used by a user (you) to construct any arbitrary cohort that can be fed into downstream predictive modelling pipelines. 

Concretely, at the end of a cohort construction you will have a table where each row represents an observation (or example) and has **at least** the following four colummns. 
1. `anon_id` : the de-identified id for a patient associated with this observation.  
2. `observation_id` : an id unique to the observation (note you may have multiple observations for the same patient)
3. `index_time` : a timestamp that marks the time at which your prediction would have been made for the patient for this observation.
4. `label` : if binary then 1 indicates positive label 0 indicates negative label.  If multiclass this takes a categorical value. 

If you have a multi-label task, that is you have multiple outcomes that are not mutually exclusive (which we will demonstrate in this workshop), then your cohort table will have multiple "label" columns. 

Additionally you may find it useful to append additional information to a cohort table (example demographic variables or additional metadata about your labels/outcomes).  Adding these columns shouldn't break my code...  

### Constructing the Cohort Table
As a user you have flexibility in how you choose to construct your cohort table.  If the logic to construct your cohort is simple enough, you may be able to generate the table with a small/medium sized SQL query. When you can, I recommend doing this.  As is not infrequently the case, the logic needed to construct your cohort can get quite complex.  Other times, you may require merging datasets (some maybe not on bigquery) together to create your cohort table.  In both of these scenarios, it becomes favorable to apply additional python logic to construct your final cohort table. The `CohortBuilder` class defined in `medinfo.dataconversion.cohorts` allows this flexibility. 

#### 1.1 Import the cohorts module and instantiate a CohortBuilder Object 
The `CohortBuilder` class provides skeleton logic for you to construct any arbitrary cohort table.  Given a SQL query and an optional python function, it creates a cohort and saves it to a user specified table in our bigquery project. Specifically, it's constructor takes in: 
1. a dataset name
2. a table name
3. a list of label columns (names of the columns that correspond to outcomes in your cohort)

Execute the code below to import the cohorts module and create an instance of `CohortBuilder` that will save a cohort table titled `{your_sunetid}_devworkshop_cohort` in the `devworkshop` dataset within the `mining-clinical-decisions` project. We'll pass in a list of strings to populate the `label_columns` attribute.  Our SQL query to generate this table will then be tailored such that the resulting table contains columns listed in `label_columns`.   

In [4]:
import cohorts
sunet_id='[TODO]' ### TODO Fill in 
cb = cohorts.CohortBuilder(dataset_name='devworkshop',
                           table_name=f'{sunet_id}_devworkshop_cohort',
                           label_columns=['label_WBC', 'label_HCT', 'label_PLT'])

#### 1.2 Develop a SQL query to generate your cohort table
This step is often one of the most nuanced.  In this workshop we'll be generating a cohort of patients who have had CBC w/ Differential labs ordered for them. The prediction task will be to, at the moment in time the order is placed, predict whether various components (White Blood Cell, Hematocrit, and Platelets) will return outside of the normal range. This is a **multi-label** prediction task, as outcomes for each of these three components are not mutually exclusive. We'll thus generate a cohort table that has the following columns. 
1. `anon_id`
2. `observation_id`: we'll use the order_id associated with the lab order
3. `index_time`: this will be the timestamp associated with the lab order
4. `label_WBC`: 1 if the resulting white blood cell count component falls outside the normal range, 0 otherwise.
5. `label_HCT`: 1 if the resulting hematocrit component falls outside the normal range, 0 otherwise.
6. `label_PLT`: 1 if the resulting platelets component falls outside the normal range, 0 otherwise.

In this workshop we won't restrict observations based on `anon_id` — which means the same patient may have multiple observations (examples, or rows) in the cohort table. We'll also only look at labs ordered between 2015 and 2020, and we'll randomly sample 2000 observations per year to keep the dataset lightweight (12k rows total).  Finally, we'll only make predictions for lab orders that a) result and b) result with all three components we care about.  At prediction time we may not know if the ordered lab will result (ex will it be cancelled, will only a subset of components result?), which can have implications on your estimates of model performance (think censoring and how that biases things). Limitations aside, below I've provided a skeleton SQL query with **TODOs** for ya'll to fill in so that this logic can be executed.   

In [5]:
query="""
WITH cbcd_lab_results as (
    SELECT DISTINCT
        anon_id,
        order_id_coded,
        order_time_utc as index_time,
        base_name,
        CASE WHEN result_flag is NULL OR result_flag = "Normal" Then 0
        ELSE 1
        END label
    FROM 
        shc_core.lab_result # what table in shc_core has lab results?
    WHERE 
        proc_code = 'LABCBCD' # Can we get a proc_code associated with CBC w/ Differential?
    AND
        base_name in ('WBC', 'PLT', 'HCT')
    AND 
        EXTRACT(YEAR FROM order_time_utc) BETWEEN 2015 and 2020
),

# Pivot lab result to wide
cohort_wide as (
    SELECT 
        * 
    FROM 
        cbcd_lab_results
    PIVOT (
        MAX(label) as label -- should be max of one value or no value (hence check not null)
        FOR base_name in ('WBC', 'PLT', 'HCT')
    )
    WHERE 
        -- only keep labs where all three components we care about result
        label_WBC is not NULL and label_PLT is not NULL and label_HCT is not NULL
)

### 2000 observations randomly sampled per year
SELECT 
    anon_id, order_id_coded as observation_id, index_time, label_WBC, label_PLT, label_HCT
FROM 
     (SELECT *,
             ROW_NUMBER() OVER  (PARTITION BY EXTRACT(YEAR FROM index_time) ORDER BY RAND()) 
             AS seqnum
      FROM cohort_wide 
     ) 
WHERE 
    seqnum <= 2000

"""

#### 1.3 Generate your cohort and save to bigquery. 
You can optionally pass in a table schema to control the datatypes of each column.  Here we force `anon_id` to be a string and `observation_id` to be an integer as an example. 

In [6]:
cb.build_cohort(query) # Executes query, and stores cohort in a pandas dataframe in the `df` attribute
schema=[{'name' : 'anon_id', 'type' : 'STRING'},
        {'name' : 'observation_id', 'type' : 'INTEGER'}]
cb.write_cohort_table(overwrite='True', schema=schema) # Writes the cohort to bigquery

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5817.34it/s]


####  1.4 [Optional] Additional notes
If you'd like to further process your table using python logic you may implement an additional function that takes in a pandas dataframe and outputs a pandas dataframe. You can then pass this function into the `build_cohort` method as an argument: `transform`. By default the `transform` argument takes in an identity function, that is it takes in a dataframe and returns the same dataframe. Example below.

In [7]:
def other_cohort_logic(df):
    """
    Here we'll just return the same dataframe, but you can imagine writing any other custom code to transform
    the result of your SQL query in any way shape or form. You don't even need to use the result
    of the query at all if you don't want to you.  You could for example read in a CSV external
    from bigquery that has all of your cohort information and return that as a dataframe.  The
    result of this function is what gets written to bigquery and becomes your final cohort. 
    """
    return df

cb.build_cohort(query, transform=other_cohort_logic) # stores cohort in `df` attribute
schema=[{'name' : 'anon_id', 'type' : 'STRING'},
        {'name' : 'observation_id', 'type' : 'INTEGER'}]
cb.write_cohort_table(overwrite='True', schema=schema) # Writes the cohort to bigquery

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 3048.19it/s]


### 2. Featurization
Once you have a cohort table saved with appropriate columns on bigquery, the next step becomes extracting features for each of your observations.  You can dive deep into crazy rabbit holes generating custom features for any given prediction task.  That's not the point of this workshop. Here we'll use a standard cookie cutter featurization approach with various feature types found in EHR data that people commonly use for a wide array of prediction tasks. 

#### Feature Represention
As a primer, we'll first categorize feature types seen in structured EHR data into two buckets. These are 1) categorical features [these are features that do not have corresponding values - ex an ICD code] and 2) numerical features [these features have numerical information tied to them - ex a lab result, a vital sign, age]. 

We'll use feature types from both of these categories in our lab prediction models. We'll represent features from both categories as counts in "bag of words" fashion.  For categorical features, this is trivial.  Each feature (think ICD code, lab order, medication order) will be represented with its own column in the resulting feature matrix.  The value for a particular observation will be the number of times that feature is present in the patient record starting at `index_time` and looking back some user defined amount of time.

For numerical features, representing as counts (bag of words) requires an additional set of transformations.  We need some mechanism of tokenizing numerical features before we can treat them as counts.  To do this, we will bin each feature into discrete buckets with thresholds defined as percentile cutoffs from the distribution of the feature we see in the training set. The number of buckets for a given feature is user defined. As an example, let's say we want to tokenize a temperature measurement.  In a pre-processing step, we'll build a distribution of temperature measurements using all measurements found in the training set. We'll create thresholds for each feature based on the percentile values in the training set distribution.  If we specify to create 5 buckets, then values that land in the 0-20th percentile get placed in bucket 1 ... 80-100th percentile becomes bucket 5. Each bucket for each numerical feature then becomes a column in the final feature matrix, and the value is, like before, the number of times a value in that bucket is present in the patient timeline starting at `index_time` and looking back some user defined amount of time.

#### Splitting
The `featurizer` module also defines a splitting mechanism — the mechanism at which you will split your data into training and test sets. By default we do a time split where the last year of data is held out as the test set. This mimics the deployment scenario where you train on prior data and deploy on future data. Model decay due to dataset drift will be accounted for in your model performance estimates — though clearly dataset drift can be more or less of a factor depending on the time window of your study / deployment. 

#### Storing a feature matrix
Our bag of words feature representation will result in a wide and sparse feature matrix. Saving this in wide format is innefficient. By default your feature matrix will be saved to bigquery in long form, and will be saved to your working directory as [csr matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).  The featurizer module never actually stores your feature matrix as a table in wide format. This is all abstracted away for you, but what's nice about this representation is you can fit matrices that would otherwise use ungodly amounts of RAM into memory on a standard computer. Sklearn models support csr matrices, which is also quite nice.

### 2.1 Import the featurizer module and instantiate a BagOfWordsFeaturizer
The constructor takes in the following arguments 
1. cohort_table : name of the cohort table you saved to bigquery in prior step
2. label_columns : names of label columns in cohort table (same as before)
3. dataset_name : name of dataset on bigquery with your cohort table
4. table_name : name of feature matrix table you will save back to bigquery (in long form)
5. outpath : a local (your computer) path you want to save feature matrices and labels to
6. feature_config : optionally define a custom featurization schema - more explained in 2.2

Edit the code below to instantiate a BagOfWordsFeaturizer object that points to your cohort table.  Save your long form feature matrix in a bigquery table called `{your_sunet}_feature_matrix` in the `devworkshop` dataset. Save in a local directory of your choosing. 

In [8]:
import featurizers

your_sunetid='[TODO]' # TODO
featurizer = featurizers.BagOfWordsFeaturizer(
  cohort_table=f'{your_sunetid}_devworkshop_cohort', # TODO Your cohort table
  label_columns=['label_WBC', 'label_HCT', 'label_PLT'],
  dataset_name='devworkshop',
  table_name=f'{your_sunetid}_feature_matrix',
  outpath='./model_info/' # edit however you want - this path will be created if it doesn't already exist. 
)

### 2.2 Call your featurizer object to generate a feature matrix. 
Feature types, number of bins, and look back windows are all customizable. Here however you'll use the default. The default featurization schema is below. 

```json
DEFAULT_DEPLOY_CONFIG = {
    'Categorical': {
        'Sex': [{'look_back' : None}],
        'Race': [{'look_back' : None}],
        'Diagnoses': [{'look_back' : None}],
        'Medications': [{'look_back' : 28}]
    },
    'Numerical': {
        'Age': [{'look_back': None, 'num_bins': 5}],
        'LabResults': [{'look_back': 14, 'num_bins': 5}],
        'Vitals': [{'look_back': 3, 'num_bins': 5}]
    }
}
```

You'll notice the first level classifies feature types into categorical and numerical features. Categorical features include sex, race, diagnoses, and medications. Numerical features include, age, lab results, and vital signs.

Categorical features have specified one or more look back windows. When a look back window is not appropriate (ie sex, race) it's defined as None. We specificy a look back window of None for diagnoses to indicate that by default we look back over the entire patient timeline for prior diagnoses. For medications, we look back 28 days. 

Numerical features have one or more look back windows and number of bins assigned to them. In the default config, we will featurize lab results available 14 days prior to index time, and we will bin each lab result into 5 buckets. For vital signs, we look back three days from index time. 

**Observation**
You'll notice the name of this particular config has the word DEPLOY.  Models trained using this specification can easily be silently deployed into an epic production environemnt (perhaps a future group meeting).  

Execute the code below to generate and save your feature matrix. Check your `outpath` afterwords to see what gets dumped in there.

**Disclaimer** 
Messing around too much with this config may break the code in it's current state. You can subtract feature types, change look back windows, and number of bins — but having multiple look back windows and number of bins for a single feature type is currently not supported. 


In [9]:
featurizer() # this just calls the featurizer

Featurizing sex
Featurizing race
Featurizing diagnosis codes
Featurizing medication orders
Featurizing age


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8905.10it/s]


Featurizing lab results


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 8612.53it/s]


Featurizing Vitals


Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████| 12000/12000 [00:00<00:00, 12711.61rows/s]


Feature matrix generated with 16670 features


### 3. Model Training
The final part of this workshop entails actually training and evaluating models for the lab prediction task. There are many nuances that come up with model training and model selection. For our purposes here we will follow a minimally viable procedure. Our task is again **multi-label**, meaning our outcomes are not mutually exclusive.  To model this, we will train three separate binary classifiers. One classifier will predict the hematocrit component, one white blood cell, one platelets. We'll train and evaluate a random forest using default hyperparamters in sklearn.  Again — clearly way more you could do with model selection, that's not the point of this workshop.  

#### 3.1 Import the trainers module and instantiate a BaselineModelTrainer
`BaselineModelTrainer` defines logic to train a minimally viable classifier that can get you some *lower* bound on the predictive performance associated with your task of interest. It implements one of the simplest supervised learning pipelines imaginable — it trains and evalutes a random forest with default hyperparameters. The constructor takes in one argument — the path to the directory you saved your feature matrices. In the cell below, write code to import the trainers submodule and instantiate a `BaselineModelTrainer`

In [10]:
import trainers
path='./model_info/' # TODO where are your feature matrices stored?
trainer = trainers.BaselineModelTrainer(path)

#### 3.2 Call your BaselineModelTrainer to train a random forest
By calling your newly created object (just like you called your featurizer), you will execute logic to train a random forest for a given `task`.  Each task is defined by it's corresponding `label_column`: ie (label_WBC, label_HCT, label_PLT).  

Upon being called a `BaselineModelTrainer` will print out an AUC (area under the ROC curve) using predictions and labels from your test set (which by default is your last year of data). A dataframe containing the columns `predictions` and `labels` is written to a csv in the same directory containing your feature matrix.  You can load in this information later to further evaluate your model however you see fit (generate ROC curves, precision recall curves, calibration plots, etc). More on this at a later time, but on top of this all necessary information needed to **deploy** the model in an epic environment is stored in a `deployment_config` file. This includes
1. The model itself,
2. The order of features in an observations feature vector
3. The binning thresholds for all numerical features
4. The feature config file indicating which feature types to pull in and transform using EPIC/FHIR apis. 

Execute the code below to train and evaluate three random forests (one for each lab component). 


In [11]:
# Component 1 - Predict white blood cell count
trainer(task='label_WBC')

AUC: 0.74


In [12]:
# Component 2 - Predict hematrocrit
trainer(task='label_HCT')

AUC: 0.81


In [13]:
# Component 3 - Predict platelets
trainer(task='label_PLT')

AUC: 0.8


### [Optional] 3.3 Further analyze performance of your models
A CSV containing predictions and labels associated with each of these three tasks has been written to your specified local path. Read in these CSV's and try generating some ROC and Precision recall curves.  Documentation about how to do that is [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) and [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html). 

In [ ]:
### TODO: generate three ROC and Precision recall curves.

### [Optional] 4. Train a baseline model for a task of your choosing
Done already?  Try following the same three step procedure to train and evaluate a baseline model for a task of your choosing.  Is there a particular cohort of patients you have in mind and a potentially intersting/meaningful prediction task that could benefit patients like them?  Are you in the midst of on ongoing ML project?  Try repeating this workflow for your task of interest. 

In [ ]:
### TODO, repeat workflow for your task of interest

### [Optional] 5: Contributing to this workflow
If you found this workflow helpful and want to make it even better - there are many ways we can think about improving.  As of now the code you all executed is fairly barebones. Only models built against the `shc_core` extract are supported, the only feature representation supported is bag of words, and the only modelling infrastucture is defined in `BaselineModelTrainer`.  Below I list some additional features (overloaded term I know) that if supported would make this infrastucture better. If ya'll have time, feel free to claim one of these feature requests for yourself.  We can talk offline about how to go about integrating into existing infrastructure. 

**CohortBuidler**
* Would be good to support functionality that does not require you to write your cohort table back to bigquery.

**Featurizers**
* Support inclusion of different feature types found in the EHR ex orders in order_proc (including lab orders, procedure orders, microbiology orders, imaging orders etc) as well as other features from various other tables we have available
* Support different feature representations
    * TF-IDF representation (instead of bag of words)
    * SVD reprsentation (reduce dimensionality of TF-IDF feature vector with SVD for final representation)
    * Summary stats representation (still treat categorical variables as counts, but take summary stats ex mean, std, first, last values over continuious features instead of binning) -- this is what was supported in the original FeatureMatrixFactory module built for the HealthRex group.
    * Timeline representation: feature vectors don't have to be fixed length to be fed into RNN style model classes. When using these kinds of models we could represent each observation as a timeline of tokens. 
* Support feature selection (ie remove low and high IC features or features with low variance prior to modelling). 

**Modelling**
* Support a model selection pipeline that includes hyperparamter sweeps. I have code for this, just have yet to plug in...
* Support pytorch models and model trainers. 
    * Functionality to easily train FFNNs, GRUs, transformers would be beneficial and would be nice to plug into model selection workflow. 
  
**Supporting other data sources**
* lpch_core is an easy first target as structure to shc_core is nearly identical.  
* Supporting OMOP would be more challenging but likely worth it, especially for people who want to include features from clinical notes.  

**Any other ideas welcomed! -- Thanks for taking part of this workshop!**